# Trabajo de Fin de Máster: Applicación Web sobre un modelo de Question Answering en español

    
Descripción de los objetivos del proyecto.
- Imagen de Bert
- ¿Cómo aprenderá Bert?
- ¿Cómo disponibilizaremos la app final?

---
## Capítulo 0: Configuración Inicial

### Importación de librerias

Descripción

In [1]:
import numpy as np
import os
import sys
ABS_DIR = os.path.join(os.getcwd(), "..")
sys.path.append(ABS_DIR)


import utils.read_and_write as rw
import utils.preprocesado as pp
import train.train_utils as tu
import predict.predict_utils as pu

### Configuración del proyecto

Desde la siguiente URL: https://huggingface.co/mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es

* Descargamos los ficheros de entrenamiento y evalución de SQuAD en español
* Descargamos los ficheros de vocab y config con los que se entrenó BETO (Spanish BERT)

In [2]:
CONFIG_DIR = os.path.join(ABS_DIR, "config\\")

DATA_DIR = os.path.join(ABS_DIR, "data\\")
LOG_DIR = os.path.join(DATA_DIR, "logs\\")
MODELS_DIR = os.path.join(DATA_DIR, "models\\")
DATASETS_DIR = os.path.join(DATA_DIR, "datasets\\")
CHECKPOINTS_CALLBACK_DIR = os.path.join(DATA_DIR, "checkpoints\\")
CACHE_DIR = os.path.join(DATA_DIR, "cache\\")

logger = rw.crear_logger("tfm-app.log")

train_path = rw.comprobar_fichero_existe(os.path.join(DATASETS_DIR, "train-v2.0-es.json"), logger)
eval_path = rw.comprobar_fichero_existe(os.path.join(DATASETS_DIR, "dev-v2.0-es.json"), logger)

vocab_BERT_path = rw.comprobar_fichero_existe(os.path.join(MODELS_DIR, "vocab.txt"), logger)
config_BERT_path = rw.comprobar_fichero_existe(os.path.join(MODELS_DIR, "config.json"), logger)

config_file = rw.cargar_config()

2021-10-14 23:03:45,307 tfm-app.log INFO: El fichero log tfm-app.log ha sido creado
2021-10-14 23:03:45,310 tfm-app.log INFO: El fichero train-v2.0-es.json existe en D:\tfm_app\src\notebooks\..\data\datasets
2021-10-14 23:03:45,311 tfm-app.log INFO: El fichero dev-v2.0-es.json existe en D:\tfm_app\src\notebooks\..\data\datasets
2021-10-14 23:03:45,312 tfm-app.log INFO: El fichero vocab.txt existe en D:\tfm_app\src\notebooks\..\data\models
2021-10-14 23:03:45,314 tfm-app.log INFO: El fichero config.json existe en D:\tfm_app\src\notebooks\..\data\models


### Visualización de datos

Explicación de cómo son los datos

In [3]:
train_df = rw.json_to_dataframe(train_path)
print(train_df.shape)
eval_df = rw.json_to_dataframe(eval_path)
print(eval_df.shape)

(86818, 6)
(9905, 6)


* ENTRENAMIENTO

In [4]:
id_row = 5000
print(f"Título:\n {train_df.iloc[id_row, 1]}")
print("---"*30)
print(f"Párrafo:\n {train_df.iloc[id_row, 2]}")
print("---"*30)
print(f"Pregunta:\n {train_df.iloc[id_row, 3]}")
print("---"*30)
print(f"Posicion de respuesta:\n {train_df.iloc[id_row, 4]}")
print("---"*30)
print(f"Respuesta:\n {train_df.iloc[id_row, 5]}")

Título:
 Budismo
------------------------------------------------------------------------------------------
Párrafo:
 La segunda verdad es que el origen de la dukkha puede ser conocido. En el contexto de las cuatro verdades nobles, el origen de dukkha se explica comúnmente como deseo (Pali: Tanha) condicionado por la ignorancia (Pali: Avijja). En un nivel más profundo, la causa raíz del dukkha se identifica como ignorancia (Pali: Avijja) de la verdadera naturaleza de las cosas. La tercera verdad noble es que el cese completo de la dukkha es posible, y la cuarta verdad noble identifica un camino a este cese. [Nota 7]
------------------------------------------------------------------------------------------
Pregunta:
 El origen del dukkha se explica como deseo condicionado por qué?
------------------------------------------------------------------------------------------
Posicion de respuesta:
 201
------------------------------------------------------------------------------------------

* VALIDACIÓN

In [5]:
id_row = 5000
print(f"Título:\n {eval_df.iloc[id_row, 1]}")
print("---"*30)
print(f"Párrafo:\n {eval_df.iloc[id_row, 2]}")
print("---"*30)
print(f"Pregunta:\n {eval_df.iloc[id_row, 3]}")
print("---"*30)
print(f"Posicion de respuesta:\n {eval_df.iloc[id_row, 4]}")
print("---"*30)
print(f"Respuesta:\n {eval_df.iloc[id_row, 5]}")

Título:
 Escuela privada
------------------------------------------------------------------------------------------
Párrafo:
 Las escuelas religiosas afiliadas y denominacionales forman una subcategoría de escuelas privadas. Algunas de estas escuelas enseñan educación religiosa, junto con los sujetos académicos habituales para impresionar las creencias y tradiciones de su fe particular en los estudiantes que asisten. Otros usan la denominación como una etiqueta más general para describir lo que los fundadores basaron en su creencia, manteniendo una fina distinción entre académicos y religión. Incluyen escuelas parroquiales, un término que a menudo se utiliza para denotar escuelas católicas romanas. Otros grupos religiosos representados en el sector de la educación privada K-12 incluyen protestantes, judíos, musulmanes y cristianos ortodoxos.
------------------------------------------------------------------------------------------
Pregunta:
 Junto con musulmanes, judíos y cristianos pr

---
## Capítulo 1: Preprocesado de datos

### Configurando el tokenizador

In [6]:
tokenizador = pp.obtener_tokenizador(vocab=vocab_BERT_path, lowercase=False)

In [7]:
tokens = tokenizador.encode("Esto es una prueba para ver cómo se tokeniza")
print(f'ids: {tokens.ids}')
print(f'tokens: {tokens.tokens}')

ids: [4, 2098, 1058, 1108, 3889, 1110, 1292, 2078, 1062, 3, 5]
tokens: ['[CLS]', 'Esto', 'es', 'una', 'prueba', 'para', 'ver', 'cómo', 'se', '[UNK]', '[SEP]']


### Modificando la configuración

In [8]:
import json
with open(config_BERT_path, 'r') as json_file:
    config_BERT = json.load(json_file)
config_BERT.update(config_file["train"]["preprocess"])

### Preprocesamiento

* Entrenamiento

In [9]:
x_train, y_train, train_dataset, train_squad_objects, train_errors = pp.transformar_datos_squad(train_df, tokenizador, config_BERT, logger=logger, name_data="entrenamiento")

2021-10-14 23:05:02,925 tfm-app.log INFO: Transformado el conjunto de entrenamiento al formato SquadExample
2021-10-14 23:05:08,854 tfm-app.log INFO: Se ha conseguido obtener los inputs y los targets para el conjunto de entrenamiento. Se han creado  85629 puntos


* Validación

In [10]:
x_eval, y_eval, eval_dataset, eval_squad_objects, eval_errors = pp.transformar_datos_squad(eval_df, tokenizador, config_BERT, logger=logger, name_data="validación")

2021-10-14 23:05:16,230 tfm-app.log INFO: Transformado el conjunto de validación al formato SquadExample
2021-10-14 23:05:16,905 tfm-app.log INFO: Se ha conseguido obtener los inputs y los targets para el conjunto de validación. Se han creado  9649 puntos


---
## Capítulo 2: Construcción del modelo

### Hiperparámetros

In [11]:
config_BERT

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 31002,
 'max_seq_len': 384,
 'lr': 0.05,
 'batch_size': 64,
 'nb_epoch': 20}

### Callbacks

In [12]:
callbacks = tu.generar_callbacks(x_eval, y_eval, eval_squad_objects, logger=logger)

### Formato Keras

In [13]:
train_dataset_keras = pp.input_formato_keras(train_dataset, config_BERT)

### Modelo

In [14]:
modelo = tu.obtener_modelo(logger=logger, config=config_BERT)

Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFBertForQuestionAnswering: ['mlm___cls']
- This IS expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2021-10-14 23:05:19,370 tfm-app.log INFO: Se ha realizado la carga del modelo

In [15]:
modelo.summary()

Model: "tf_bert_for_question_answering"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109850880 
_________________________________________________________________
qa_outputs (Dense)           multiple                  1538      
Total params: 109,852,418
Trainable params: 1,538
Non-trainable params: 109,850,880
_________________________________________________________________


## Capítulo 3: Entrenamiento y validación

### Monitorización en Tensorboard

In [16]:
# Cargamos el tensorboard
%load_ext tensorboard
# %reload_ext tensorboard

In [17]:
%tensorboard --logdir '..\\data\\logs\\tensorboard\\'

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 2028.

### Entrenamiento

In [18]:
modeloHistory = tu.entrenar_modelo(modelo, train_dataset_keras, config_BERT, callbacks, logger=logger)

2021-10-14 23:06:33,247 tfm-app.log INFO: Se realizará el entrenamiento con 20 epochs


Epoch 1/20
   1/1338 [..............................] - ETA: 0s - loss: 11.8802 - output_1_loss: 5.9133 - output_2_loss: 5.9669WARNING:tensorflow:From d:\python_entornos\tf_gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
  51/1338 [>.............................] - ETA: 1:55:54 - loss: 8.0869 - output_1_loss: 4.0228 - output_2_loss: 4.0641

KeyboardInterrupt: 

## Capítulo 4: Predicción sobre el modelo entrenado

### Carga del modelo

In [ ]:
modelo = rw.cargar_modelo(logger=logger)

In [ ]:
modelo.summary()

## Predicción

In [ ]:
ejemplo_id = 0
context = eval_df.loc[0, "Context"]
print("Contexto:")
print(context, "\n")

question = eval_df.loc[0, "Question"]
print("Pregunta:")
print(question, "\n")

respuesta = eval_df.loc[0, "Text"]
print("Respuesta:")
print(respuesta)

In [ ]:
os.path.join(MODELS_DIR, "tfm_bert_finetuned_squadV2esp\\v1")

In [ ]:
modelo.load_weights(os.path.join(MODELS_DIR, "tfm_bert_finetuned_squadV2esp\\v1\\saved_model.pb"))

In [ ]:
all_results = []
for count, inputs in enumerate(eval_dataset.batch(16)):
    x, _ = inputs  
    start_logits, end_logits = modelo(x, training=False)
    output_dict = dict(
        start_logits=start_logits,
        end_logits=end_logits)
    for result in get_raw_results(output_dict):
        all_results.append(result)
    if count % 100 == 0:
        print("{}/{}".format(count, 2709))

In [ ]:
pred_start, pred_end = modelo.predict(eval_dataset.batch(16))

In [ ]:
texto = "Esto es una texto de prueba.\n Vamos a ver si funciona"

In [ ]:
print(texto)

In [ ]:
pu.whitespace_split(texto)

In [ ]:
doc_tokens = ["E"]

In [ ]:
doc_tokens[-1] += "s"

In [ ]:
doc_tokens

In [ ]:
for w in texto:
    print(w)